In [1]:
# !pip install peft
# !pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.4 MB/s eta 0:00:0000:0100:01


In [2]:
# from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
# import torch 

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16
# )
# base_model = 'model-attribution-challenge/bloom-2b5'
# model = AutoModelForCausalLM.from_pretrained(base_model,
#                                              quantization_config=bnb_config,
#                                              torch_dtype=torch.float16,
#                                              device_map='auto'
#                                             )
# tokenizer = AutoTokenizer.from_pretrained(base_model)


config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [3]:
# tokenizer.all_special_ids

In [4]:
# model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2560)
    (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear4bit(in_features=2560, out_features=7680, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2560,), eps=1e-05, elementwise_af

In [5]:
# #QLoRA
# from peft import get_peft_model, LoraConfig, PeftModel, prepare_model_for_kbit_training

# model = prepare_model_for_kbit_training(model)
# config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     bias='none',
#     lora_dropout=0.05,
#     task_type='CAUSAL_LM',
#     target_modules=['dense', 'query_key_value'],
# )

# lora_model = get_peft_model(model, config)

In [6]:
# #sô lượng tham số cần train
# lora_model.print_trainable_parameters()

trainable params: 7,372,800 || all params: 3,009,930,240 || trainable%: 0.2449


In [7]:
# pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [8]:
# from datasets import Dataset, concatenate_datasets

# paths = ['/kaggle/input/data-qa-python-06102004/Data-QA-Python.csv', '/kaggle/input/data-qa-py/data-gpt-create-6-10-2024.csv', '/kaggle/input/datasetqapy/python_questions_answers.csv']
# df = []
# for path in paths:
#     data = Dataset.from_csv(path)
#     try:
#         df.append(data.rename_columns({'Question': 'question', 'Answer':'answer'}))
#     except:
#         df.append(data)

# df = concatenate_datasets(df).shuffle()

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
# df['question']

In [12]:
# import time

# def tokenizer_function(example):
#     prompt = ['<s>[INST]' + val['question'] + ' ?  \n[/INST] ' + val['answer'] + ".</s>" for val in example]
#     x = [key for key, i in enumerate(prompt) if len(tokenizer.tokenize(i)) >= 600 and i[-1] != '?']
#     print(len(x))
#     y = [i for i in example['answer'] if i[-1] == '?']
#     print(len(y))
#     # print(x)
#     prompt = [i for i in prompt if len(tokenizer.tokenize(i)) < 600]
#     example = [i for key, i in enumerate(example['answer']) if key not in x]
#     print(example[:1])
#     inputs = tokenizer(prompt, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
#     labels = tokenizer(example, padding='max_length', truncation=True, max_length=600, return_tensors='pt')
#     inputs['labels'] = labels['input_ids']
#     return inputs

# st  = time.time()
# token_df = tokenizer_function(df)
# print(time.time() - st)

14
121
['Bạn có thể sử dụng từ khóa in để kiểm tra xem một phần tử có tồn tại trong tuple hay không.']
10.063823699951172


In [13]:
# token_df = Dataset.from_dict(token_df)

In [14]:
# token_df.set_format(type='torch', columns=['input_ids', 'attention_mask'])
# type(token_df['input_ids'])

torch.Tensor

In [15]:
# token_df[:1]

In [16]:
# from sklearn.model_selection import train_test_split

# train , eval_ = train_test_split(token_df, test_size=0.20, random_state=42)

# train = Dataset.from_dict(train)
# train.set_format(type='torch', columns=['input_ids', 'attention_mask'])
# print(train)

# eval_ = Dataset.from_dict(eval_)
# eval_.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [ ]:
# from transformers import TrainingArguments, Trainer

# agrs = TrainingArguments(
#     output_dir = './result',
#     per_device_train_batch_size=4,
#     # per_device_eval_batch_size=1,
#     # gradient_accumulation_steps=12,
#     learning_rate=2e-4,
#     weight_decay=3e-4,
#     eval_strategy='steps',
#     eval_steps=2000,
#     logging_dir='./loss',
#     # logging_steps=100,
#     num_train_epochs=3,
#     fp16=True,
# )

# trainer = Trainer(
#     model=lora_model,
#     args=agrs,
#     train_dataset=token_df,
#     # eval_dataset=eval_,
#     data_collator=data_collator,
# )

# trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


In [ ]:
# dcu
# import tensorflow as tf

# loss_tr = []
# loss_eval = []
# for event in tf.compat.v1.train.summary_iterator('/kaggle/working/loss/events.out.tfevents.1731751471.64a4e43df86f.30.0'):
#     for value in event.summary.value:
#         if 'train/loss' in value.tag:
#             loss_tr.append([event.step, value.simple_value])
#         elif 'eval/loss' in value.tag:
#             loss_eval.append([value.simple_value])
            

In [ ]:
# import pandas as pd

# df = pd.DataFrame(l, columns=['step','train','eval'])

In [ ]:
# df.to_csv("loss.csv")

In [ ]:
#lưu adapter QloRA
# lora_model.save_pretrained('adapter', save_adapter=True,save_config =True)

In [1]:
pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.8 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("model-attribution-challenge/bloom-2b5")
tokenizer = AutoTokenizer.from_pretrained('model-attribution-challenge/bloom-2b5', return_tensors='pt', padding='max-length', max_length=512)

# model_lora = PeftModel.from_pretrained(model, '/content/ada-qlora') #chay tren gg colab  
model_lora = PeftModel.from_pretrained(model, '/kaggle/working/ada-qlora') #chay tren kaggle 
# model.merge_and_unload()

# model.save_pretrained('merged-model')

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
import time

def input_sent(prompt):
    # while True:
      # prompt = input("Bạn: ")
      prompt = prompt[0].upper() + prompt[1:]
      prompt = prompt.strip()
      st = time.time()
      if prompt.lower() == 'exit':
        return
      prompt = '<s>[INST]' + prompt + ' ?  \n[/INST] '
      # model_lora.to('cuda')
      inputs = tokenizer(prompt, return_tensors='pt')
      output = model_lora.generate(
          **inputs,
#           max_length=50,
          min_length=10,
          max_new_tokens=512,
          # top_k = 10,
          top_p = 0.95,
          do_sample=True,
          repetition_penalty=1.05,
          
        )
      print(tokenizer.decode(output[0], skip_special_tokens=True).replace('\\n', '\n').replace('\\t', '\t')) 
      print(time.time() - st)


In [4]:
questions = [
    "1. Python là gì? Tại sao Python được sử dụng phổ biến?",
    "2. Làm thế nào để cài đặt Python trên máy tính?",
    "3. Biến trong Python là gì? Cách khai báo biến?",
    "4. Sự khác biệt giữa Python 2 và Python 3 là gì?",
    "5. Các kiểu dữ liệu cơ bản trong Python là gì?",
    "6. Làm thế nào để tạo một list và thêm phần tử vào list?",
    "7. Sự khác biệt giữa list và tuple trong Python?",
    "8. Lambda function là gì? Khi nào nên sử dụng nó?",
    "9. Sự khác biệt giữa `==` và `is` trong Python là gì?",
    "10. Làm thế nào để đọc và ghi file trong Python?",
    "11. Exception handling trong Python là gì? Sử dụng `try`, `except`, `finally` như thế nào?",
    "12. Làm thế nào để định nghĩa một hàm (function) trong Python?",
    "13. Làm thế nào để sử dụng vòng lặp `for` và `while` trong Python?",
    "14. List comprehension là gì? Tại sao nên sử dụng nó?",
    "15. Sự khác biệt giữa `break`, `continue` và `pass` trong vòng lặp?",
    "16. Module và package trong Python là gì? Làm thế nào để sử dụng chúng?",
    "17. Sự khác biệt giữa hàm `append()` và `extend()` của list?",
    "18. Làm thế nào để cài đặt và sử dụng các thư viện bên ngoài bằng `pip`?",
    "19. Decorator trong Python là gì? Khi nào nên sử dụng decorator?",
    "20. Sự khác biệt giữa mutable và immutable trong Python?"
]

for idx, question in enumerate(questions, start=1):
    input_sent(question)

# input_sent()

UnboundLocalError: local variable 'prompt' referenced before assignment

In [ ]:
# pip install pyngrok

In [ ]:
# #tạo api để backend call đến

# from flask import Flask, request, jsonify
# from pyngrok import ngrok
# import time 

# app = Flask(__name__)

# @app.route('/', methods=['POST'])
# def predict():
#   st = time.time()
#   data = request.get_json()['input']
#   data = "Câu hỏi: " + data + " ?\n\nCâu trả lời: "
#   inputs = tokenizer(data, return_tensors='pt').to('cuda')
#   model_lora.to('cuda')
#   out = model_lora.generate(
#       **inputs,
#       # top_p = 0.85,
#       # top_k = 5,
#       # do_sample=True,
#       # repetition_penalty = 1.05,
#       max_new_tokens=80,
#   )
#   response = tokenizer.decode(out[0], skip_special_tokens=True).strip()
#   end = time.time()
#   print(response, end - st)
#   return jsonify({
#       'response':response.replace('\\n', '\n').replace("\\t", "\t").split('endtoken')[0]
#   })

# if __name__ == '__main__':
#   ngrok.set_auth_token("thay bằng token của tk ngrok")
#   ngrok_tunnel = ngrok.connect(5000)
#   print('Public URL:', ngrok_tunnel.public_url)#url api đây
#   app.run()
